# We simulate real proxies (without the filters) and realistic SNR (=1)

* We use 50 speleothems and 50 icecores <- added the option 'how_many'
* Yearly
* snr=1
* 10 repetitions with frac=0.8

In [1]:
cfg = {
#BASIC PARAMS
'vp':{
'd18O':'/home/mchoblet/paleoda/data/model_data/downsampled/ECHAM5_d18O_850_1849.nc',
'evap':'/home/mchoblet/paleoda/data/model_data/downsampled/ECHAM5_evap_850_1849.nc',
'prec':'/home/mchoblet/paleoda/data/model_data/downsampled/ECHAM5_prec_850_1849.nc',
'tsurf':'/home/mchoblet/paleoda/data/model_data/downsampled/ECHAM5_tsurf_850_1849.nc'},
'oro': None,
'reconstruct':['tsurf','d18O','prec'], 
'output_folder':'experiment_Echam', #in cwd
'output_file':'Echam_realistic_PPE_snr1',
'multi_model_prior': None, #None or a list where the first entry is a dictionary of dictionaries for the file paths and secord a dictionary with the orography files
'avg':None, 
'seed': 42, 
'check_nan':False, #mainly needed for iHadCM3, because of its missing years nans are introduced with 'resampled'
###PROXYS
#'obsdata':['/home/mchoblet/paleoda/data/proxy_dbs/sisal_db.nc'],#['/home/mchoblet/paleoda/data/proxy_dbs/sisal_db.nc'],#,'/home/mchoblet/paleoda/data/proxy_dbs/iso2k_db.nc'],
'obsdata':['/home/mchoblet/paleoda/data/proxy_dbs/sisal_1600_1800.nc','/home/mchoblet/paleoda/data/proxy_dbs/iso2k_db_1600_1800.nc'],
'obs_var':[['d18O','d18O_precision'],['d18O','d18O_precision']],#,['d18O','d18O_precision']], #name of variables and uncertainties in proxy_db files
'how_many': [50,50], #option to decide from the beginning how many proxies from each database are used. Meant to be used for Pseudoproxy experiment where number of proxies is relevant. One entry for each DB
'proxy_error': [0.1,0.1], #irrelevant for PPE
'time_scales': ['median'], 
'proxy_time':['0850','1849'], 
'resample': False,#True,
'resample_mode': 'linear',#'nearest',
'mask': 3, ))
###MULTI-TIMESCALE
#'timescales':[1,5,10,20,40], #list of timescales (have to be true divisors of longest timescale (block size)) Timescale 1 always required! 
#'timescales':[1],
'timescales':[1],
'reuse': True,#True, #e.g if a yearly resolved proxy is also to be used in the 5,10 year mean reconstruction
###PSM
'psm':['speleo','icecore'], 
'interpol':None, #None/'dw'
'var_psm': ['d18O','d18O'] ,
'speleo':{
    'weighting': 'inf',#'inf', #None(pure d18O) / prec (d180 prec weighted)/ inf (d18O prec-evap weighted)
    'height': False, #height correction: True requires orography of model + proxy_db must have a coordinate 'elev'. Lapse rate assumed -0.28 for isotopes and -0.65 for temperature
    'fractionation': True, #requires tsurf (in 'vp')
    'fractionation_temp': 'regular', #'regular'
    'filter': False, #True: convolution of yearly means with 2.5 year transit time, or float: transit time
    't_time': 2.5, #if filter equal true, use this as a transit time
},
'icecore':{
    'weighting':'prec', #None(pure d18O) / prec (d180 prec weighted)
    'height': False, #height correction: True requires orography of model and 'elev' coordinate for proxy_db. Requires that proxy_db has a coordinate 'lapse rate' (e.g. i attributed -0.15 to non polar porxies and -0.28 to polar ones (per 100m)
    'filter': False, #If True the PSM-function from PRYSM which takes into account diffusion is applied. 
},
#currently this only works for one Database files, probably at some point that needs to be more flexible
'linear':{
    'variable': 'tsurf' , #variable to regress against
    'type': 'anom' , #absolute value or anomaly
    'anom_time': None , #None for full simulation, or time-list 
    'uniform': None, # if you just want one linear formula to rule them all [slope, intercept]
    #names of the regression parameters in the proxy-db
    'slope': 'slope' ,
    'intercept': 'intercept',
    'error': 'error', #make sure that the error in the database info is really the regression error *variance*, not standard error.
},
###PSEUDOPROXY-EXPERIMENTS (variable comes from var_psm)
'ppe': {
    'use': True,
    'SNR': [1,1], #signal to noise variance ratio
    'noise_type': 'w', #'w'/'r'
    'red_noise_a': 0.32, #autocorrelation for red noise
    'multi_timescale': [[1],[1]], 
    'source': 'internal', #internal/external
    'other_model': 
    {'vp':{'tsurf':'/home/ldap-server/draco/cleaned_data/final/CESM_tsurf_850_1850.nc',
    'prec':'/home/ldap-server/draco/cleaned_data/final/CESM_prec_850_1850.nc',
    'd18O':'/home/ldap-server/draco/cleaned_data/final/CESM_d18O_850_1850.nc',
    'evap': '/home/ldap-server/draco/cleaned_data/final/CESM_evap_850_1850.nc'}},
    'oro': '/home/ldap-server/draco/orographies/final/icesm_oro.nc',
    'metrics_ppe': ['Corr','p_value','p_bootstrap','eff_p_value','CE','RE',],#'RMSE','RMSPE','MAE','MAPE'],
    'psm' : {
    'speleo': {'weighting':'inf', 'height': False, 'fractionation':True, 'fractionation_temp': 'regular', 'filter': False, 't_time': 2.5,},
    'icecore':{'weighting':'prec', 'height': False, 'filter': False,},
    },
},
    # cfg['ppe']['psm']= {'speleo':None, 'icecore': None} # if the config from above for the psm (which gives the obs-from-model) shall be different for the pseudoproxies.
#else just leave it as {'speleo':None, 'icecore': None} 
    
###KALMAN-FILTER
'time': ['0850','1849'], #covered by all models
'nens':100, #ensemble members
'reps':10, #Monte Carlo
'full_fields': True, 
'proxy_frac': 0.8, #choose 100 random proxies
'extra_assi': 
    {
        'globalmean': ['tsurf','d18O','prec'], #None or list of variables
        'lat_mean': None, #['tsurf'], #None or list of variables
        'regionalmean': None, #None or list of variables
        'region':None, #None or list of list [[latS,latN],[latW,latE]] #specify the regions in 0-360 longitude system
        'prec_weighted': False, #if you want to get the precipitation weighted d18O
    },
'error_scale':1, 
'anomaly': None, #'BG+HX', #/ 'HX'
'anomaly_time': None, #['1600','1800'],#None,#['1500','1750'],#None,
'cov_loc': None,
#evaluation metrics (which do not rely on a true state)
'metrics': [None],#['rank_histogram','rank_histogram_posterior','CD'], #['None']
}

In [2]:
import sys
sys.path.append('/home/mchoblet/paleoda')

import xarray as xr
Masayoshi Takanaka
import dataloader
import tqdm
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import cartopy.feature as cfeature  
from matplotlib.offsetbox import AnchoredText    
import utils

import copy

%load_ext autoreload
%autoreload 2

In [27]:
wrapper.paleoda_wrapper(cfg)

>>>>>LOADING PROXY DATA
Load d18O  from  /home/mchoblet/paleoda/data/model_data/downsampled/ECHAM5_d18O_850_1849.nc
computing yearly average
Load evap  from  /home/mchoblet/paleoda/data/model_data/downsampled/ECHAM5_evap_850_1849.nc
computing yearly average
Load prec  from  /home/mchoblet/paleoda/data/model_data/downsampled/ECHAM5_prec_850_1849.nc
computing yearly average
Load tsurf  from  /home/mchoblet/paleoda/data/model_data/downsampled/ECHAM5_tsurf_850_1849.nc
computing yearly average
USING SPELEO PSM
>>>>>>>>>>>>>GETTING MONTHLY d18O Data
>>>>>>>>>>>>>APPLYING FRACTIONATION
use time-varying temperature
USING ICECORE PSM
GENERATING PSEUDOPROXIES FROM SAME MODEL
>>>>>>>>>>>>GENERATING pseudoproxies with SNR: 1
>>>>>>>>>>>>GENERATING pseudoproxies with SNR: 1
Start Multitimescale DA loop.


100%|██████████| 10/10 [01:48<00:00, 10.88s/it]


Finished multitimescale DA
Save variables
Compute pseudoproxy metrics for  tsurf
Compute pseudoproxy metrics for  d18O
Compute pseudoproxy metrics for  prec
Output written to Echam_realistic_PPE_snr1.nc


<xarray.Dataset>
Dimensions:            (lat: 48, lon: 96, time: 1000, model: 1, reps: 10,
                        nens: 100, site: 100, bs: 1, time_res_1: 1000,
                        site_1: 100)
Coordinates:
  * lat                (lat) float64 87.16 83.48 79.78 ... -79.78 -83.48 -87.16
  * lon                (lon) float64 0.0 3.75 7.5 11.25 ... 348.8 352.5 356.2
  * time               (time) object 0850-01-01 00:00:00 ... 1849-01-01 00:00:00
    MC_idx             (model, reps, nens) int64 428 920 977 444 ... 716 985 102
  * site               (site) <U5 '0.85' '0.99' '0.86' ... '1.57' '1.146'
  * time_res_1         (time_res_1) object 0850-01-01 00:00:00 ... 1849-01-01...
  * site_1             (site_1) <U5 '0.85' '0.99' '0.86' ... '1.57' '1.146'
Dimensions without coordinates: model, reps, nens, bs
Data variables: (12/35)
    tsurf_mean         (time, lat, lon) float64 256.3 256.4 ... 229.0 228.8
    tsurf_std          (time, lat, lon) float64 0.6906 0.6899 ... 0.5771 0.5738
    tsurf_Corr         (lat, lon) float64 0.4595 0.4605 0.4578 ... 0.6008 0.5999
    tsurf_p_value      (lat, lon) float64 2.131e-53 1.207e-53 ... 8.75e-99
    tsurf_p_bootstrap  (lat, lon) float64 101.0 101.0 101.0 ... 101.0 101.0
    tsurf_eff_p_value  (lat, lon) float64 2.131e-53 1.207e-53 ... 8.75e-99
    ...                 ...
    gm_tsurf_std       (time) float64 0.1471 0.1471 0.1471 ... 0.1471 0.1471
    gm_d18O            (time) float64 -6.253 -6.261 -6.306 ... -6.21 -6.218
    gm_d18O_std        (time) float64 0.04306 0.04306 ... 0.04306 0.04306
    gm_prec            (time) float64 84.37 84.56 84.05 ... 84.46 84.7 84.45
    gm_prec_std        (time) float64 0.4225 0.4225 0.4225 ... 0.4225 0.4225
    proxies_res_1      (time_res_1, site_1) float64 -4.247 -10.56 ... -18.66
Attributes:
    prior:    [{'long_name': 'delta of total precipitation', 'units': 'o/oo',...
    cfg:      {'vp': {'d18O': '/home/mchoblet/paleoda/data/model_data/downsam...

# Plots 
- (2 x 3 map plot (RE and CE are the same))



In [5]:
sisal=xr.open_dataset('/home/mchoblet/paleoda/data/proxy_dbs/sisal_1600_1800.nc')
iso=xr.open_dataset('/home/mchoblet/paleoda/data/proxy_dbs/iso2k_db_1600_1800.nc')

In [7]:
lats=np.concatenate([sisal.lat,iso.lat])
lons=np.concatenate([sisal.lon,iso.lon])

In [ ]:
###

path='/home/mchoblet/paleoda/results/experiments/experiment_Echam/Echam_realistic_PPE_snr1.nc'
results=xr.open_dataset(path)

sns.set_theme(style="white")

metrics=['Corr','CE']

#get first list item
conf=0

figsize=(int(15*1.2),int(6*1.2))
fig, axes = plt.subplots(ncols=3,nrows=2,figsize=figsize,subplot_kw={'projection':ccrs.PlateCarree()})

vmin=-1
vmax=1

color='grey'
size=5
ms='x'

for i,m in enumerate(metrics):
    
    #temperature    
    string='tsurf'+'_'+m
    p1=results[string].plot(ax=axes[i,0],vmin=vmin,vmax=vmax,add_colorbar=False,cmap=plt.cm.RdBu_r)
    
    ax=axes[i,0]
    ax.axes.xaxis.set_visible(True)
    ax.axes.yaxis.set_visible(True)
    ax.axes.xaxis.set_ticks([])
    ax.axes.yaxis.set_ticks([])

    ax.add_feature(cfeature.COASTLINE)
    ax.axes.gridlines(color='black',alpha=0.5,linestyle='--')
    
    ax.set_ylabel(m,fontsize=30,rotation=0)
    ax.set_xlabel('',fontsize=30,rotation=0)
    ax.yaxis.set_label_coords(-0.14,0.45)
    
    
    ax.scatter(lons,lats,c=color,s=size, markerstyle='x',transform=ccrs.PlateCarree())
    
    #d18O
    string='d18O'+'_'+m
    p2=results[string].plot(ax=axes[i,1],vmin=vmin,vmax=vmax,add_colorbar=False,cmap=plt.cm.RdBu_r)

    ax=axes[i,1]
    ax.axes.xaxis.set_visible(True)
    ax.axes.yaxis.set_visible(True)
    ax.axes.xaxis.set_ticks([])
    ax.axes.yaxis.set_ticks([])

    ax.add_feature(cfeature.COASTLINE)
    ax.axes.gridlines(color='black',alpha=0.5,linestyle='--')
    
    ax.set_ylabel('',fontsize=50,rotation=0)
    ax.set_xlabel('',fontsize=30,rotation=0)

    ax.scatter(lons,lats,c=color,s=size, transform=ccrs.PlateCarree())
    
    #precipitation
    string='prec'+'_'+m
    p3=results[string].plot(ax=axes[i,2],vmin=vmin,vmax=vmax,add_colorbar=False,cmap=plt.cm.RdBu_r)
    
    ax=axes[i,2]
    ax.axes.xaxis.set_visible(True)
    ax.axes.yaxis.set_visible(True)
    ax.axes.xaxis.set_ticks([])
    ax.axes.yaxis.set_ticks([])

    ax.add_feature(cfeature.COASTLINE)
    ax.axes.gridlines(color='black',alpha=0.5,linestyle='--')
    
    ax.set_xlabel('',fontsize=30,rotation=0)
    ax.set_ylabel('',fontsize=30,rotation=0)

    ax.scatter(lons,lats,c=color,s=size, transform=ccrs.PlateCarree())
    
p_0 = axes[0,2].get_position().get_points().flatten()
p_1 = axes[1,2].get_position().get_points().flatten()

ax_cbar = fig.add_axes([p_1[2]+0.01, p_1[1], 0.01, p_0[3]-p_1[1]])
cbar=fig.colorbar(p1, cax=ax_cbar, orientation='vertical')
cbar.ax.tick_params(labelsize=25)

"""    

p_1 = axes[-1,1].get_position().get_points().flatten()
p_2 = axes[-1,2].get_position().get_points().flatten()

ax_cbar = fig.add_axes([p_0[0], 0.1, p_0[2]-p_0[0], 0.01])
cbar=fig.colorbar(p1, cax=ax_cbar, orientation='horizontal')
#cbar=fig.colorbar(p1,ax=axes[-1,0],extend='max',aspect=12,pad=0.02,location='bottom')
cbar.ax.tick_params(labelsize=25)

ax_cbar = fig.add_axes([p_1[0], 0.1, p_1[2]-p_1[0], 0.01])
cbar=fig.colorbar(p2, cax=ax_cbar, orientation='horizontal')
cbar.ax.tick_params(labelsize=25)

ax_cbar = fig.add_axes([p_2[0], 0.1, p_2[2]-p_2[0], 0.01])
cbar=fig.colorbar(p3, cax=ax_cbar, orientation='horizontal')
cbar.ax.tick_params(labelsize=25)
"""

"""
cm=plt.cm.RdBu_r    
ax_cbar1 = fig.add_axes([p1[0], 0.01, (p1[2]-p1[0]), 0.01])
cb=fig.colorbar(p1, cax=ax_cbar1, orientation='horizontal',cmap=cm)
cb.ax.tick_params(labelsize=20)
tick_locator = ticker.MaxNLocator(nbins=3)
cb.locator = tick_locator
cb.update_ticks()
"""    

#cb.set_label('$\overline{{Proxy}}$ - $\overline{Simulation}$',fontsize=30)

axes[0,0].set_title('Tsurf',fontsize=30)
axes[0,1].set_title('$\delta^{18}O$',fontsize=30)
axes[0,2].set_title('Prec',fontsize=30)

plt.savefig('/home/mchoblet/paleoda/results/plots/thesis_draft/echam_snr1.pdf', format='pdf',bbox_inches='tight')

#plt.tight_layout() 
    